# 1.4.1多进程
### 1.使用os模块中的fork方式实现多进程
#### fork方法是调用一次，返回两次，操作系统将当前进程（父进程）复制出一份子进程，这两个进程几乎完全相同。子进程中永远返回0，父进程返回子进程的id（在linux系统中才有效）

In [4]:
import os
if __name__ == '__main__':
    print('currnet Process (%s) start...'%(os.getpid()))
    pid = os.fork()
    if pid < 0:
        print('error in fork')
    elif pid == 0:
         print('i am a child process(%s) and my parent process is(%s)' % (os.getpid(), os.getppid()))
    else:
        print('I(%s) created a child process(%s).' % (os.getpid(), pid))

currnet Process (15664) start...


AttributeError: module 'os' has no attribute 'fork'

### 2.使用multiprocessing模块创建多进程

In [14]:
import os
from multiprocessing import Process
# 子进程要执行的代码
def run_proc(name):
    print('child process %s (%s) running...' % (name, os.getpid()))
if __name__ == '__main__':
    print('parent process %s' % os.getpid())
    for i in range(5):
        p = Process(target=run_proc, args=(str(i), ))
        print('process will start.')
        p.start()
    p.join()
    print('process end')

parent process 1284
process will start.
process will start.
process will start.
process will start.
process will start.
process end


### 3.multiprocessing提供了一个Pool类来代表进程池对象

In [ ]:
from multiprocessing import Pool
import os, time, random

def run_task(name):
    print('task %s (pid=%s) is running...' % (name, os.getpid()))
    time.sleep(random.random()*3)
    print('task %s end.' % name)

if __name__ == '__main__':
    print('current process %s' % os.getpid())
    p = Pool(processes=3)
    for i in range(5):
        p.apply_async(run_task, args=(i, ))
    print('waiting for all subprocesses done...')
    p.close()
    p.join()
    print('all subprocesses done.')

current process 1284
waiting for all subprocesses done...


# 注：多进程在pycharm下正常工作

# 1.4.2 多线程
#### 多线程类似于同时执行多个不同程序，多线程运行有如下优点：
#### 1.可以把运行时间长的任务放到后台去处理。
#### 2.用户界面可以更加吸引人，比如用户点击了一个按钮去触发某些事件的处理，可以弹出一个进度条来显示处理的进度
#### 3.程序的运行速度可能加快
#### 4.在一些需要等待的任务实现上，如用户输入、文件读写和网络收发数据等，线程就比较有用了。在这种情况下我们可以释放一些珍贵的资源，如内存占用等

## 1.用threading模块创建多线程

In [1]:
import random
import time, threading
# 新线程执行的代码:
def thread_run(urls):
    print('Current %s is running...' % threading.current_thread().name)
    for url in urls:
        print('%s ---->>> %s' % (threading.current_thread().name, url))
        time.sleep(random.random())
    print('%s ended.' % threading.current_thread().name)

print('%s is running...' % threading.current_thread().name)
t1 = threading.Thread(target=thread_run, name='Thread_1', args=(['url1', 'url2', 'url3'], ))
t2 = threading.Thread(target=thread_run, name='Thread_2', args=(['url4', 'url5', 'url6'], ))
t1.start()
t2.start()
t1.join()
t2.join()

MainThread is running...
Current Thread_1 is running...Current Thread_2 is running...

Thread_1 ---->>> url1Thread_2 ---->>> url4

Thread_1 ---->>> url2
Thread_2 ---->>> url5
Thread_1 ---->>> url3
Thread_1 ended.
Thread_2 ---->>> url6
Thread_2 ended.


### 第二种方式从threading.Thread继承创建线程类，将方法一进行重写：

In [4]:
import random
import threading
import time
class myThread(threading.Thread):
    def __init__(self,name,urls):
        threading.Thread.__init__(self,name=name)
        self.urls = urls
    
    def run(self):
        print('current %s is running...' % threading.current_thread().name)
        for url in self.urls:
            print('%s --->>> %s' % (threading.current_thread().name,url))
            time.sleep(random.random())
        print('%s ended.' % threading.current_thread().name)       
print('current %s is running...' % threading.current_thread().name)
t1 = myThread(name='Thread_1',urls=['url_1','url_2','url_3'])
t2 = myThread(name='Thread_2',urls=['url_4','url_5','url_6'])
t1.start()
t2.start()
t1.join()
t2.join()
print('%s ended.' % threading.current_thread().name)

current MainThread is running...
current Thread_1 is running...
Thread_1 --->>> url_1
current Thread_2 is running...
Thread_2 --->>> url_4
Thread_2 --->>> url_5
Thread_2 --->>> url_6
Thread_1 --->>> url_2
Thread_1 --->>> url_3
Thread_1 ended.
Thread_2 ended.
MainThread ended.


## 2.线程同步
#### 如果多个线程共同对某个数据修改，则可能出现不可预料的结果，为了保证数据的正确性，需要对多个线程进行同步。使用Thread对象的Lock和RLock可以实现简单的线程同步，这两个对象都有acquire方法和release方法，对于那些每次只允许一个线程操作的数据，可以将其操作放到acquire和release方法之间